In this step we are filtering the tweets of the Key Opinion Leaders, that we retrived in the first step. 


In [1]:
from twitter_crawler import TwitterCrawler

my_token = 'AAAAAAAAAAAAAAAAAAAAADRFOwEAAAAAaTp%2Bdd1OhobYMYb5ExPXm7IL6RA%3DDHknH402gOXoegUGxNtpC6giIjdackRLtdRx6tjrnnLeFN1ntT'
my_twitter_crawler = TwitterCrawler(bearer_token= my_token)


In [2]:
### inputs for the filter function
import os 
dir_with_all_tweets = os.path.join("KOL tweets")

score_for_KOP = 5
score_for_key_event = 5
score_for_key_words = 5

key_words = ["brexit", "eu", "deal", "economy"]
threshold_score = 10

KOP_excel_name = "KOP brexit.xlsx"
key_events_excel_name = "Brexit_key_events.xlsx"
stop_words_to_add = ["https"]
#########################################################
tweets_table_filtered, tweets_table, csv_files_evaluated = my_twitter_crawler.filter_tweets_Brexit(
    dir_with_all_tweets = dir_with_all_tweets,
    score_for_KOP = score_for_KOP,
    score_for_key_event = score_for_key_event,
    score_for_key_words = score_for_key_words,
    key_words = key_words,
    threshold_score = threshold_score,
    KOP_excel_name = KOP_excel_name,
    key_events_excel_name = key_events_excel_name,
    stop_words_to_add = stop_words_to_add,
    stop_words_file_name = "stopwords.txt",
    verbose = True)

Step 1: Reading all the csv files
Step 2: Reading The Events Excel File
Step 3: Reading and preprocessing Key Opinion Leaders data
Step 4: Reading stop words data and adding your stop words

Step 5: Preprocess the tweets table for scoring:
Removing Stop words        | bigram, trigram, forthgram | merging the tweets table with the KOP and Events tables | 
found 111331 bigrams
found 106829 trigram
found 88204 forthgram

Finish preprocessing the tweets table, it took: 146.029 seconds
Step 6: Scoring the Tweets

Finish!
Total time: 187.687 Seconds ( 3.128 Minutes)


In [3]:
print(f"Total number of tweets of the KOL={tweets_table.shape[0]}")
print(f"Total number of filtered tweets of the KOL={ tweets_table.shape[0] - tweets_table_filtered.shape[0]}")
print(f"Total number of remained tweets of the KOL={tweets_table_filtered.shape[0]}")

Total number of tweets of the KOL=191682
Total number of filtered tweets of the KOL=176757
Total number of remained tweets of the KOL=14925


In [2]:
import pandas as pd

tweets_table_filtered = pd.read_excel('tweets_table_filtered.xlsx')

In [5]:
tweets_table_filtered['conversation_id'] = tweets_table_filtered['conv_id_new'].apply(lambda x: x.replace("conv_id: ", ""))


In [9]:
top_n_tweets = list(tweets_table_filtered.head()['conversation_id'])
top_n_tweets

['859743329952706560',
 '1072239308554362881',
 '920968477275512835',
 '836951115090563072',
 '859817622120083456']

In [10]:
path_for_table_dict = my_twitter_crawler.return_comments_by_tweet_ids(conversation_ids = top_n_tweets, query = "",
                              start_time = "2014-01-01T00:00:00Z",
                              end_time = "today",
                              max_results = 100, evaluate_last_token = False,
                              limit_amount_of_returned_comments = 500000,
                              verbose = False, dir_tree_name = "Step 3 Comments")

Bringing comments of 859743329952706560
The dir Step 3 Comments already exist
creating directory Step 3 Comments\conv_tree_for_859743329952706560 to insert all the comments of the given conversation-id
creating directory Step 3 Comments\conv_tree_for_859743329952706560\log_comments_for_conversation_id_859743329952706560 to insert all the logs of the comments for the tweet id -  859743329952706560
no more comments from this conversation id
Total amount of collected comments =  2
2
---------------------------------------------------------------
Bringing comments of 1072239308554362881
The dir Step 3 Comments already exist
creating directory Step 3 Comments\conv_tree_for_1072239308554362881 to insert all the comments of the given conversation-id
creating directory Step 3 Comments\conv_tree_for_1072239308554362881\log_comments_for_conversation_id_1072239308554362881 to insert all the logs of the comments for the tweet id -  1072239308554362881
1 Got from twitter 97 comments, and there are 

In [16]:
TwitterCrawler.get_url_by_tweet_id(tweet_id = "1072239308554362881")

'https://twitter.com/anyuser/status/1072239308554362881'

In [42]:
import os
 
directory = 'Step 3 Comments'

all_comments_df_list = []
for root, dirs, files in os.walk(directory):
    for filename in files:
        if not filename.endswith('csv'): continue
        file_path = os.path.join(root, filename)
        comments_table =  pd.read_csv(file_path)
        all_comments_df_list.append( comments_table)

all_comments_df = pd.concat(all_comments_df_list)

In [43]:
all_comments_df['conversation_id'] = all_comments_df['conversation_id'].apply(lambda x : str(x))

In [44]:
type(list(all_comments_df['conversation_id'])[0])

str

In [45]:
merged_comments = pd.merge(all_comments_df,tweets_table_filtered,left_on='conversation_id',right_on='conversation_id' , how="left")

,Unnamed: 0_x,author_id_x,author_id_new_x,conv_id_new_x,conversation_id,created_at_x,entities.mentions_x,id_x,id_new_x,in_reply_to_user_id,...,Name,Born_in,twitter_user_name,Role,Place,clean_user_name,score_key_words,score_KOP,total_score,conv_id_new1
0,0,213044641,author_id: 213044641,coversation_id: 1072239308554362881,1072239308554362881,2018-12-18T12:42:57.000Z,"[{'start': 0, 'end': 12, 'username': 'anthonyz...",1075008513909366785,id: 1075008513909366785,15662428,...,Charles GRANT,England,@CER_Grant,Director,Centre for European Reform,CER_Grant,3,1,20,1072239308554362881
1,1,92599000,author_id: 92599000,coversation_id: 1072239308554362881,1072239308554362881,2018-12-15T08:32:24.000Z,"[{'start': 0, 'end': 16, 'username': 'charles_...",1073858301102866432,id: 1073858301102866432,14292921,...,Charles GRANT,England,@CER_Grant,Director,Centre for European Reform,CER_Grant,3,1,20,1072239308554362881
2,2,795921978758729728,author_id: 795921978758729728,coversation_id: 1072239308554362881,1072239308554362881,2018-12-15T07:40:53.000Z,"[{'start': 0, 'end': 12, 'username': 'anthonyz...",1073845334391050240,id: 1073845334391050240,15662428,...,Charles GRANT,England,@CER_Grant,Director,Centre for European Reform,CER_Grant,3,1,20,1072239308554362881
3,3,14292921,author_id: 14292921,coversation_id: 1072239308554362881,1072239308554362881,2018-12-15T02:16:03.000Z,"[{'start': 0, 'end': 15, 'username': 'danielmg...",1073763587729080321,id: 1073763587729080321,938591299,...,Charles GRANT,England,@CER_Grant,Director,Centre for European Reform,CER_Grant,3,1,20,1072239308554362881
4,4,14292921,author_id: 14292921,coversation_id: 1072239308554362881,1072239308554362881,2018-12-15T02:13:40.000Z,"[{'start': 0, 'end': 15, 'username': 'danielmg...",1073762986353926145,id: 1073762986353926145,14292921,...,Charles GRANT,England,@CER_Grant,Director,Centre for European Reform,CER_Grant,3,1,20,1072239308554362881
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122,25,1049168464890482688,author_id: 1049168464890482688,coversation_id: 1072239308554362881,1072239308554362881,2018-12-10T21:24:35.000Z,"[{'start': 0, 'end': 14, 'username': 'European...",1072240687767322624,id: 1072240687767322624,956890108493533185,...,Charles GRANT,England,@CER_Grant,Director,Centre for European Reform,CER_Grant,3,1,20,1072239308554362881
123,26,2499402907,author_id: 2499402907,coversation_id: 1072239308554362881,1072239308554362881,2018-12-10T21:24:09.000Z,"[{'start': 0, 'end': 10, 'username': 'CER_Gran...",1072240575720669186,id: 1072240575720669186,634496564,...,Charles GRANT,England,@CER_Grant,Director,Centre for European Reform,CER_Grant,3,1,20,1072239308554362881
124,27,784967425,author_id: 784967425,coversation_id: 1072239308554362881,1072239308554362881,2018-12-10T21:21:13.000Z,"[{'start': 0, 'end': 10, 'username': 'CER_Gran...",1072239836998852608,id: 1072239836998852608,634496564,...,Charles GRANT,England,@CER_Grant,Director,Centre for European Reform,CER_Grant,3,1,20,1072239308554362881
125,0,21111724,author_id: 21111724,coversation_id: 859743329952706560,859743329952706560,2017-05-03T12:45:53.000Z,"[{'start': 0, 'end': 14, 'username': 'MartinSe...",859750862180290561,id: 859750862180290561,2421073909,...,Martin SELMAYR,Germany,@MartinSelmayr,Head of Cabinet of the President,of the European Commission European Commission,MartinSelmayr,2,1,20,859743329952706560
